In [58]:
import requests
import pandas as pd
import time

In [59]:
key = '72e4097c974d4ddebaa4a2a2674d70d3'
endpoint = "https://api.cognitive.microsoft.com/bing/v7.0/search"
headers = {"Ocp-Apim-Subscription-Key": key}
params = {"q": '', "textDecorations": False, "textFormat": "HTML", 'count': 1,'responseFilter':'Webpages',}

In [60]:
MA_public_df = pd.read_csv('MA_public_schools.csv')
MA_private_df = pd.read_csv('MA_private_schools.csv')

In [61]:
test_schools_20 = pd.concat([MA_public_df.sample(n=10),MA_private_df.sample(n=10)]).astype('object')
test_schools_20 = test_schools_20.reset_index().drop('index',axis = 1)
test_schools_20.to_csv("MA_test_schools_20.csv")

In [62]:
test_schools_20 = pd.read_csv('MA_test_schools_20.csv', index_col = 0)

In [63]:
url_filters = ['facebook.com','elementaryschools.org','usnews.com','greatschools.org'
              'niche.com','schooldigger.com','wikipedia.org']

In [64]:
#This currently has an 85% success rate
def name_and_school(row):
    search_term = row['Org Name']
    if ':' in search_term:
        search_term = search_term.split(':')[1]
    if 'school' not in search_term.lower():
        search_term += ' School'
    return search_term

Might be improved by filtering out websites that are not correct and taking the first link not white listed.
We could also try looking at those white listed sites to get the link from them as they often have that information but we would need to be careful.

In [65]:
def get_url(row,searchTermFunc):
    params['q'] = searchTermFunc(row)
    response = requests.get(endpoint, headers=headers, params=params)
    search_results = response.json()
    url = search_results['webPages']['value'][0]['url']
    #To prevent calling the API too many times per second
    time.sleep(.5)
    return url

In [66]:
#Be careful running this as it performs as many API calls as the test_schools_df and we have limited amounts of calls
test_schools_20['url'] = test_schools_20.apply(get_url,args = [name_and_school],axis=1)

In [67]:
test_schools_20.head()

,Org Name,Org Type,Address 1,Town,State,Zip,url
0,Methuen: Methuen High,Public School,1 Ranger Rd,Methuen,MA,1844,http://www.methuen.k12.ma.us/
1,Winthrop: Winthrop Middle School,Public School,60 Payson Street,Winthrop,MA,2152,https://www.winthrop.k12.ma.us/
2,Dudley-Charlton Reg: Shepherd Hill Regional High,Public School,68 Dudley Oxford Road,Dudley,MA,1571,http://www.dcrsd.org/schools/shepherd-hill-reg...
3,Sandwich: Sandwich STEM Academy,Public School,365 Quaker Meetinghouse Road,Sandwich,MA,2537,https://www.sandwichk12.org/schools/stem-academy
4,Westford: Westford Academy,Public School,30 Patten Road,Westford,MA,1886,https://www.niche.com/k12/westford-academy-wes...


In [68]:
for i,row in test_schools_20.iterrows():
    print(row['Org Name'],row['url'])

Methuen: Methuen High http://www.methuen.k12.ma.us/
Winthrop: Winthrop Middle School https://www.winthrop.k12.ma.us/
Dudley-Charlton Reg: Shepherd Hill Regional High http://www.dcrsd.org/schools/shepherd-hill-regional-high-school
Sandwich: Sandwich STEM Academy https://www.sandwichk12.org/schools/stem-academy
Westford: Westford Academy https://www.niche.com/k12/westford-academy-westford-ma/
Pittsfield: Morningside Community School http://morningside.pittsfield.net/
Woburn: Linscott-Rumford https://www.woburnpublicschools.com/linscott/
Melrose: Roosevelt http://www.lyndhurstschools.net/school_home.aspx?schoolid=6
Reading: Wood End Elementary School http://www.reading.k12.ma.us/woodend/
Lowell: Lowell High https://www.lowell.k12.ma.us/lowelllhs
North Shore Christian School - Beacon Campus http://nschristian.org/
Saint Pius X School https://www.stpiusbowie.org/
Bradford Christian Academy https://bradfordchristianacademy.org/
Acera School https://www.aceraschool.org/
St Bernadette School h